In [7]:
# !pip install azure-storage-blob

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 392.2/392.2 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.3/194.3 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.2 MB/s eta 0:00:00


In [8]:
import requests
import pandas as pd
from datetime import datetime, timedelta
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import json

apiBase = 'https://query2.finance.yahoo.com'
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64)"
}

def getCredentials(cookieUrl='https://fc.yahoo.com', crumbUrl=apiBase+'/v1/test/getcrumb'):
    cookie = requests.get(cookieUrl).cookies
    crumb = requests.get(url=crumbUrl, cookies=cookie, headers=headers).text
    return {'cookie': cookie, 'crumb': crumb}

def get_historical_data(symbol, start_date, end_date, credentials):
    url = f"{apiBase}/v8/finance/chart/{symbol}"
    params = {
        'period1': int(start_date.timestamp()),
        'period2': int(end_date.timestamp()),
        'interval': '1d',
        'events': 'history',
        'crumb': credentials['crumb']
    }
    response = requests.get(url, params=params, cookies=credentials['cookie'], headers=headers)
    data = response.json()

    if 'chart' in data and 'result' in data['chart'] and data['chart']['result']:
        result = data['chart']['result'][0]
        dates = [datetime.fromtimestamp(ts) for ts in result['timestamp']]
        prices = result['indicators']['quote'][0]
        df = pd.DataFrame({
            'Date': dates,
            'Open': prices['open'],
            'High': prices['high'],
            'Low': prices['low'],
            'Close': prices['close'],
            'Volume': prices['volume']
        })
        return df
    else:
        print(f"No data available for {symbol}")
        return None

def upload_data_to_blob_storage(df, connection_string, container_name, blob_name):
    # Convert DataFrame to JSON
    json_data = df.to_json(orient='records', date_format='iso')

    # Create BlobServiceClient
    blob_service_client = BlobServiceClient.from_connection_string(connection_string)

    # Get ContainerClient
    container_client = blob_service_client.get_container_client(container_name)

    # Check if container exists, create if not
    if not container_client.exists():
        container_client.create_container()

    # Get BlobClient
    blob_client = container_client.get_blob_client(blob_name)

    # Upload JSON data
    blob_client.upload_blob(json_data, overwrite=True)
    print(f"Data has been successfully uploaded to {blob_name} in container {container_name}.")

def main():
    credentials = getCredentials()
    symbol = 'BANKBARODA.NS'
    end_date = datetime.now()
    start_date = end_date - timedelta(days=365)

    df = get_historical_data(symbol, start_date, end_date, credentials)

    if df is not None:
        # Replace with your actual Azure Blob Storage connection string
        connection_string = 'DefaultEndpointsProtocol=https;AccountName=storge4bob;AccountKey=GR9If90YEYUFCnTgGkAbLLJiIdMdycFBdgWFuzyO1Z3mlK14EZDOgrtedsNWHvOKeu9J2yMDxD+AStDMgfGw==;EndpointSuffix=core.windows.net'
        container_name = 'market-data'
        blob_name = 'bankbaroda_historical_data.json'

        upload_data_to_blob_storage(df, connection_string, container_name, blob_name)
        print(f"Data for {symbol} has been successfully uploaded to Azure Blob Storage.")

if __name__ == "__main__":
    main()


Data has been successfully uploaded to bankbaroda_historical_data.json in container market-data.
Data for BANKBARODA.NS has been successfully uploaded to Azure Blob Storage.
